### 计算无标度网络的阈值 $\tau=0.5$

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from igraph import *
import warnings
warnings.filterwarnings("ignore")
import copy
from joblib import Parallel,delayed

In [9]:
c=np.loadtxt('c_data.csv')
num=np.load('num.npy',allow_pickle=True).tolist()
state1=np.load('state1.npy',allow_pickle=True).tolist()
a=np.loadtxt('a_data.csv')

In [10]:
g=Graph()
# g=g.Barabasi(100,2)
g1=Graph()
# g1=g1.Barabasi(sum(c),20)
g=g.Read_Edgelist(r'Edge storage/edge_list')
g1=g1.Read_Edgelist(r'Edge storage/edge_list_info')

In [11]:
U=np.zeros([int(sum(c)),2])
A=np.zeros([int(sum(c)),2])
T=np.zeros([int(sum(c)),2])
alpha=1/14
stemp={}
for i,j in state1.items():
    stemp.update(j)
for i,j in stemp.items():
    U[i][0]=i
    U[i][1]=0.99
    A[i][0]=i
    A[i][1]=0.01
    T[i][0]=i
    T[i][1]=0
    # for i,j in stemp.items():
#     if j[1]=='U':
#         U[i][0]=i
#         U[i][1]=1
#         A[i][0]=i
#         A[i][1]=0
#         T[i][0]=i
#         T[i][1]=0
#     elif j[1]=='A':
#         U[i][0]=i
#         U[i][1]=0
#         A[i][0]=i
#         A[i][1]=1
#         T[i][0]=i
#         T[i][1]=0
#     elif j[1]=='T':
#         U[i][0]=i
#         U[i][1]=0
#         A[i][0]=i
#         A[i][1]=0
#         T[i][0]=i
#         T[i][1]=1
W=np.zeros([len(g.vs),len(g.vs)]) 
for zons in range(len(g.vs)):
    for ind in g.neighbors(zons):
        W[zons][ind]=((g.degree(ind)**(0.5))/sum([g.degree(ind)**(0.5) for ind in g.neighbors(zons) ]))
W1=np.dot(W,W.T)

In [12]:
def func1(test,tau):
    storU=[copy.deepcopy(U) for i in range(301)]
    storA=[copy.deepcopy(A) for i in range(301)]
    storT=[copy.deepcopy(T) for i in range(301)]
    for t in range(300): 
        for i,j in state1.items():
            for k,v in j.items():
                r_temp=[]
                r_tempa=[]
                for q in g1.neighbors(k):
                    r_temp.append(1-test*storA[t][q][1])
                    r_tempa.append(1-0.01*(storA[t][q][1]+storT[t][q][1]))
                storU[t+1][k][1]=storU[t][k][1]*np.prod(r_temp)+storT[t][k][1]*alpha
                storA[t+1][k][1]=storA[t][k][1]*np.prod(r_tempa)+storU[t][k][1]*(1-np.prod(r_temp))
                storT[t+1][k][1]=storT[t][k][1]*(1-alpha)+storA[t][k][1]*(1-np.prod(r_tempa))
    storU1=[]
    storA1=[]
    storT1=[]
    for i in range(100):
        storU1.append(sum((storU[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
        storA1.append(sum((storA[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
        storT1.append(sum((storT[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
    M=np.zeros([100,100])
    for i in range(len(g.vs)):
        for j in range(len(g.vs)):
            if i==j:
                M[i][j]=((((1-0.3)**2)*storU1[i]*storU1[j]+
                storU1[i]*(storA1[j]+storT1[j])*0.7*(1-0.3**2)+
                tau*(1-0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2)**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.7*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(1-0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[j][i]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.3*(1-(0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[i][j]+
                ((0.3**2)*storU1[i]*storU1[j]+
                0.3*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W1[i][j])
            elif i!=j:
                M[i][j]=((0.7*0.3*storU1[i]*storU1[j]+
                0.7*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(1-0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[j][i]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.3*(1-(0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[i][j]+
                ((0.3**2)*storU1[i]*storU1[j]+
                0.3*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W1[i][j])
    eig=np.linalg.eig(M)
    return 1/eig[0].max()

In [13]:
data5=Parallel(n_jobs=20)(delayed(func1)(test,tau=0.5) for test in np.linspace(0,0.1,11))

In [15]:
data5

[0.005108324216273608,
 0.007171706325218071,
 0.008226778808102716,
 0.00880876731934441,
 (0.009169763271848524+0j),
 0.009412714334221516,
 0.009585967844128612,
 0.009714922147429042,
 0.009814102665672347,
 0.009892380694742006,
 0.009955463957372089]

In [14]:
np.save('new_thero_value_tao5',data5)

### 无标度网络阈值为$\tau=0.3$

In [16]:
U=np.zeros([int(sum(c)),2])
A=np.zeros([int(sum(c)),2])
T=np.zeros([int(sum(c)),2])
alpha=1/14
stemp={}
for i,j in state1.items():
    stemp.update(j)
for i,j in stemp.items():
    U[i][0]=i
    U[i][1]=0.99
    A[i][0]=i
    A[i][1]=0.01
    T[i][0]=i
    T[i][1]=0
    # for i,j in stemp.items():
#     if j[1]=='U':
#         U[i][0]=i
#         U[i][1]=1
#         A[i][0]=i
#         A[i][1]=0
#         T[i][0]=i
#         T[i][1]=0
#     elif j[1]=='A':
#         U[i][0]=i
#         U[i][1]=0
#         A[i][0]=i
#         A[i][1]=1
#         T[i][0]=i
#         T[i][1]=0
#     elif j[1]=='T':
#         U[i][0]=i
#         U[i][1]=0
#         A[i][0]=i
#         A[i][1]=0
#         T[i][0]=i
#         T[i][1]=1
W=np.zeros([len(g.vs),len(g.vs)]) 
for zons in range(len(g.vs)):
    for ind in g.neighbors(zons):
        W[zons][ind]=((g.degree(ind)**(0.5))/sum([g.degree(ind)**(0.5) for ind in g.neighbors(zons) ]))
W1=np.dot(W,W.T)

In [17]:
def func1(test,tau):
    storU=[copy.deepcopy(U) for i in range(301)]
    storA=[copy.deepcopy(A) for i in range(301)]
    storT=[copy.deepcopy(T) for i in range(301)]
    for t in range(300): 
        for i,j in state1.items():
            for k,v in j.items():
                r_temp=[]
                r_tempa=[]
                for q in g1.neighbors(k):
                    r_temp.append(1-test*storA[t][q][1])
                    r_tempa.append(1-0.01*(storA[t][q][1]+storT[t][q][1]))
                storU[t+1][k][1]=storU[t][k][1]*np.prod(r_temp)+storT[t][k][1]*alpha
                storA[t+1][k][1]=storA[t][k][1]*np.prod(r_tempa)+storU[t][k][1]*(1-np.prod(r_temp))
                storT[t+1][k][1]=storT[t][k][1]*(1-alpha)+storA[t][k][1]*(1-np.prod(r_tempa))
    storU1=[]
    storA1=[]
    storT1=[]
    for i in range(100):
        storU1.append(sum((storU[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
        storA1.append(sum((storA[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
        storT1.append(sum((storT[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
    M=np.zeros([100,100])
    for i in range(len(g.vs)):
        for j in range(len(g.vs)):
            if i==j:
                M[i][j]=((((1-0.3)**2)*storU1[i]*storU1[j]+
                storU1[i]*(storA1[j]+storT1[j])*0.7*(1-0.3**2)+
                tau*(1-0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2)**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.7*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(1-0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[j][i]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.3*(1-(0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[i][j]+
                ((0.3**2)*storU1[i]*storU1[j]+
                0.3*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W1[i][j])
            elif i!=j:
                M[i][j]=((0.7*0.3*storU1[i]*storU1[j]+
                0.7*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(1-0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[j][i]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.3*(1-(0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[i][j]+
                ((0.3**2)*storU1[i]*storU1[j]+
                0.3*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W1[i][j])
    eig=np.linalg.eig(M)
    return 1/eig[0].max()

In [18]:
data3=Parallel(n_jobs=20)(delayed(func1)(test,tau=0.3) for test in np.linspace(0,0.1,11))

In [22]:
np.save('new_thero_value_tao3',data3)

### 无标度网络阈值为$\tau=0.1$

In [19]:
T=np.zeros([int(sum(c)),2])
alpha=1/14
stemp={}
for i,j in state1.items():
    stemp.update(j)
for i,j in stemp.items():
    U[i][0]=i
    U[i][1]=0.99
    A[i][0]=i
    A[i][1]=0.01
    T[i][0]=i
    T[i][1]=0
    # for i,j in stemp.items():
#     if j[1]=='U':
#         U[i][0]=i
#         U[i][1]=1
#         A[i][0]=i
#         A[i][1]=0
#         T[i][0]=i
#         T[i][1]=0
#     elif j[1]=='A':
#         U[i][0]=i
#         U[i][1]=0
#         A[i][0]=i
#         A[i][1]=1
#         T[i][0]=i
#         T[i][1]=0
#     elif j[1]=='T':
#         U[i][0]=i
#         U[i][1]=0
#         A[i][0]=i
#         A[i][1]=0
#         T[i][0]=i
#         T[i][1]=1
W=np.zeros([len(g.vs),len(g.vs)]) 
for zons in range(len(g.vs)):
    for ind in g.neighbors(zons):
        W[zons][ind]=((g.degree(ind)**(0.5))/sum([g.degree(ind)**(0.5) for ind in g.neighbors(zons) ]))
W1=np.dot(W,W.T)

In [20]:
def func1(test,tau):
    storU=[copy.deepcopy(U) for i in range(301)]
    storA=[copy.deepcopy(A) for i in range(301)]
    storT=[copy.deepcopy(T) for i in range(301)]
    for t in range(300): 
        for i,j in state1.items():
            for k,v in j.items():
                r_temp=[]
                r_tempa=[]
                for q in g1.neighbors(k):
                    r_temp.append(1-test*storA[t][q][1])
                    r_tempa.append(1-0.01*(storA[t][q][1]+storT[t][q][1]))
                storU[t+1][k][1]=storU[t][k][1]*np.prod(r_temp)+storT[t][k][1]*alpha
                storA[t+1][k][1]=storA[t][k][1]*np.prod(r_tempa)+storU[t][k][1]*(1-np.prod(r_temp))
                storT[t+1][k][1]=storT[t][k][1]*(1-alpha)+storA[t][k][1]*(1-np.prod(r_tempa))
    storU1=[]
    storA1=[]
    storT1=[]
    for i in range(100):
        storU1.append(sum((storU[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
        storA1.append(sum((storA[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
        storT1.append(sum((storT[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
    M=np.zeros([100,100])
    for i in range(len(g.vs)):
        for j in range(len(g.vs)):
            if i==j:
                M[i][j]=((((1-0.3)**2)*storU1[i]*storU1[j]+
                storU1[i]*(storA1[j]+storT1[j])*0.7*(1-0.3**2)+
                tau*(1-0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2)**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.7*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(1-0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[j][i]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.3*(1-(0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[i][j]+
                ((0.3**2)*storU1[i]*storU1[j]+
                0.3*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W1[i][j])
            elif i!=j:
                M[i][j]=((0.7*0.3*storU1[i]*storU1[j]+
                0.7*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(1-0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[j][i]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.3*(1-(0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[i][j]+
                ((0.3**2)*storU1[i]*storU1[j]+
                0.3*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W1[i][j])
    eig=np.linalg.eig(M)
    return 1/eig[0].max()

In [21]:
data1=Parallel(n_jobs=20)(delayed(func1)(test,tau=0.1) for test in np.linspace(0,0.1,11))

In [23]:
np.save('new_thero_value_tao1',data1)

### 计算ER网络下的阈值$tau=0.5$

In [24]:
g1=g1.Read_Edgelist(r'Edge storage/edge_ER_4')

In [25]:
sum(g1.degree())/len(g1.vs)

39.94095238095238

In [26]:
def func1(test,tau):
    storU=[copy.deepcopy(U) for i in range(301)]
    storA=[copy.deepcopy(A) for i in range(301)]
    storT=[copy.deepcopy(T) for i in range(301)]
    for t in range(300): 
        for i,j in state1.items():
            for k,v in j.items():
                r_temp=[]
                r_tempa=[]
                for q in g1.neighbors(k):
                    r_temp.append(1-test*storA[t][q][1])
                    r_tempa.append(1-0.01*(storA[t][q][1]+storT[t][q][1]))
                storU[t+1][k][1]=storU[t][k][1]*np.prod(r_temp)+storT[t][k][1]*alpha
                storA[t+1][k][1]=storA[t][k][1]*np.prod(r_tempa)+storU[t][k][1]*(1-np.prod(r_temp))
                storT[t+1][k][1]=storT[t][k][1]*(1-alpha)+storA[t][k][1]*(1-np.prod(r_tempa))
    storU1=[]
    storA1=[]
    storT1=[]
    for i in range(100):
        storU1.append(sum((storU[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
        storA1.append(sum((storA[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
        storT1.append(sum((storT[300][:,1][int(sum(c[:i])):int(sum(c[:i+1]))]))/c[i])
    M=np.zeros([100,100])
    for i in range(len(g.vs)):
        for j in range(len(g.vs)):
            if i==j:
                M[i][j]=((((1-0.3)**2)*storU1[i]*storU1[j]+
                storU1[i]*(storA1[j]+storT1[j])*0.7*(1-0.3**2)+
                tau*(1-0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2)**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.7*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(1-0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[j][i]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.3*(1-(0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[i][j]+
                ((0.3**2)*storU1[i]*storU1[j]+
                0.3*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W1[i][j])
            elif i!=j:
                M[i][j]=((0.7*0.3*storU1[i]*storU1[j]+
                0.7*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(1-0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[j][i]+
                (0.7*0.3*storU1[i]*storU1[j]+
                0.3*(1-(0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.7*(storA1[i]+storT1[i])*storU1[j]+
                tau*((1-0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W[i][j]+
                ((0.3**2)*storU1[i]*storU1[j]+
                0.3*((0.3)**2)*storU1[i]*(storA1[j]+storT1[j])+
                tau*(0.3**2)*0.3*(storA1[i]+storT1[i])*storU1[j]+
                tau*((0.3**2))*(0.3**2)*(storA1[i]+storT1[i])*(storA1[j]+storT1[j]))*c[j]*W1[i][j])
    eig=np.linalg.eig(M)
    return 1/eig[0].max()

In [27]:
daer5=Parallel(n_jobs=20)(delayed(func1)(test,tau=0.5) for test in np.linspace(0,0.1,11))

In [30]:
np.save('threo_er_5',daer5)

In [28]:
daer=Parallel(n_jobs=20)(delayed(func1)(test,tau=0.3) for test in np.linspace(0,0.1,11))

In [31]:
np.save('threo_er_3',daer)

In [29]:
daer1=Parallel(n_jobs=20)(delayed(func1)(test,tau=0.1) for test in np.linspace(0,0.1,11))

In [32]:
np.save('threo_er_1',daer1)